In [154]:
import numpy as np 
import pandas as pd 

In [155]:
from sklearn.cross_validation import train_test_split

DATA_FILE = 'E:/Academics/data/function_assignment/function_assignment.csv'
df = pd.read_csv(DATA_FILE,encoding='latin-1')
train, test = train_test_split(df, train_size = 0.9)


In [156]:
tags = train.function
texts = train.job_title
tags2 = test.function
texts2 = test.job_title

In [157]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from keras import metrics
print('import done')

import done


In [158]:
num_max = 1000
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)

In [159]:
mat_texts = tok.texts_to_matrix(texts,mode='count')

In [160]:
print(mat_texts[:5])

[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]]


In [161]:
print(tags.shape,mat_texts.shape)

(9130,) (9130, 1000)


In [162]:
max_len = 100
cnn_texts_seq = tok.texts_to_sequences(texts)
print(cnn_texts_seq[0])
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)
print(cnn_texts_mat[0])
print(cnn_texts_mat.shape)

[12, 1]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 12  1]
(9130, 100)


In [163]:
num_max = 1000
le2 = LabelEncoder()
tags2 = le.fit_transform(tags2)
tok2 = Tokenizer(num_words=num_max)
tok2.fit_on_texts(texts2)

In [164]:
mat_texts2 = tok.texts_to_matrix(texts2,mode='count')

In [165]:
cnn_texts_seq2 = tok.texts_to_sequences(texts2)
cnn_texts_mat2 = sequence.pad_sequences(cnn_texts_seq2,maxlen=max_len)

In [166]:
from keras.utils.np_utils import to_categorical

lab_train = to_categorical(tags)
lab_test = to_categorical(tags2)

lab_train.shape

(9130, 326)

In [167]:
from keras import optimizers
def cnn_model(): 
    model = Sequential()
    model.add(Embedding(1000,
                        20,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(256,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(326))
    model.add(Activation('sigmoid'))
    sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.5, nesterov=True)
    rms = optimizers.RMSprop(lr=0.001, rho=0.9, decay=1e-2)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics = ["accuracy"])
    return model

m = cnn_model()
m.fit(cnn_texts_mat,lab_train, batch_size = 200, nb_epoch = 10, validation_split = 0.2)

E:\Academics\Anaconda\lib\site-packages\keras\models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 7304 samples, validate on 1826 samples
Epoch 1/10
7304/7304 [==============================] - 20s 3ms/step - loss: 4.6094 - acc: 0.1083 - val_loss: 3.9895 - val_acc: 0.1292
Epoch 2/10
7304/7304 [==============================] - 18s 2ms/step - loss: 3.8989 - acc: 0.1104 - val_loss: 3.9379 - val_acc: 0.1292
Epoch 3/10
7304/7304 [==============================] - 19s 3ms/step - loss: 3.7917 - acc: 0.1006 - val_loss: 3.8030 - val_acc: 0.0827
Epoch 4/10
7304/7304 [==============================] - 18s 2ms/step - loss: 3.6428 - acc: 0.1008 - val_loss: 3.7398 - val_acc: 0.0876
Epoch 5/10
7304/7304 [==============================] - 18s 2ms/step - loss: 3.5106 - acc: 0.1015 - val_loss: 3.6067 - val_acc: 0.1391
Epoch 6/10
7304/7304 [==============================] - 19s 3ms/step - loss: 3.3993 - acc: 0.1183 - val_loss: 3.5125 - val_acc: 0.1298
Epoch 7/10
7304/7304 [==============================] - 18s 2ms/step - loss: 3.2251 - acc: 0.1777 - val_loss: 3.2827 - val_acc: 0.2448
Epoch 8